In [ ]:
! pip install pytorch-lightning~=1.5.0
! pip install datasets~=2.1.0
! pip install transformers
! pip install torchmetrics~=0.7.0
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import load_dataset
from datasets import load_metric
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

from transformers import RobertaTokenizerFast,RobertaForSequenceClassification, AdamW, TrainingArguments, Trainer,AutoModelForSequenceClassification
dataset = load_dataset("McGill-NLP/FaithDial")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',max_length=256)
itemsUnique=[]
for items in dataset["train"]["VRM"]:
    for val in items:
        if val not in itemsUnique:
            itemsUnique.append(val)

id2label = {idx:itemsUnique for idx, itemsUnique in enumerate(itemsUnique)}
label2id = {itemsUnique:idx for idx, itemsUnique in enumerate(itemsUnique)}
print(id2label)

binary_dataset = dataset.map(lambda examples: {"original_response": examples["response"]  if examples["original_response"] == None else examples["original_response"]})
import numpy as np
def preprocess_function(examples):
    encoding=tokenizer(examples["knowledge"], examples["original_response"], truncation=True,padding='max_length')
    tempLables=examples["VRM"]
    lables=np.zeros(len(label2id))
    for tempLable in tempLables:
        index=label2id[tempLable]
        lables[index]=1
    encoding["labels"]=lables
    
    return encoding
encoded_dataset = binary_dataset.map(preprocess_function,remove_columns=binary_dataset['train'].column_names)
print(dataset)

  0%|          | 0/7 [00:00<?, ?it/s]

{0: 'Disclosure', 1: 'Ack.', 2: 'Edification', 3: 'Question', 4: 'Advisement', 5: ' edification', 6: ' Advisement', 7: ' Edification', 8: ''}


  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

...

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

DatasetDict({
    test: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM'],
        num_rows: 1666
    

In [ ]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 15
VALID_BATCH_SIZE = 15
TEST_BATCH_SIZE = 15
EPOCHS = 2
LEARNING_RATE = 1e-05

In [ ]:
from transformers import default_data_collator
train_data_loader = torch.utils.data.DataLoader(encoded_dataset["train"], 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=default_data_collator
)


In [ ]:
val_data_loader = torch.utils.data.DataLoader(encoded_dataset["validation"], 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=default_data_collator
)

In [ ]:
test_data_loader = torch.utils.data.DataLoader(encoded_dataset["test"], 
    batch_size=TEST_BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=default_data_collator
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
print(len(id2label))

9


In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 9)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    for batch_idx, data in enumerate(training_loader):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['labels'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        print('after loss data in training', loss.item(), train_loss)

    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['labels'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ckpt_path = "/content/drive/MyDrive/datasets/multi-labe-two/curr_ckpt"
best_model_path = "/content/drive/MyDrive/datasets/multi-label-two/best_model.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)


############# Epoch 1: Training Start   #############
before loss data in training 1.8864401578903198 0
after loss data in training 1.8864401578903198 1.8864401578903198
before loss data in training 2.2375166416168213 1.8864401578903198
after loss data in training 2.2375166416168213 2.0619783997535706
before loss data in training 2.3894248008728027 2.0619783997535706
after loss data in training 2.3894248008728027 2.171127200126648
before loss data in training 2.743637800216675 2.171127200126648
after loss data in training 2.743637800216675 2.3142548501491547
before loss data in training 2.359372138977051 2.3142548501491547
after loss data in training 2.359372138977051 2.323278307914734
before loss data in training 3.473735809326172 2.323278307914734
after loss data in training 3.473735809326172 2.5150212248166404
before loss data in training 2.3817965984344482 2.5150212248166404
after loss data in training 2.3817965984344482 2.49598913533347
before loss data in training 1.5597364902496

In [ ]:
path = F"/content/drive/MyDrive/NLP/vrm_classification_bert.pt"

In [ ]:
torch.save(trained_model, path)

In [ ]:
import torch
trained_model = torch.load(path)
trained_model.to('cpu')
print(trained_model)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
device = 'cuda'
final_outputs=[]
for batch in test_data_loader:
  input_ids=batch['input_ids'].to(device)
  attention_mask=batch['attention_mask'].to(device)
  token_type_ids=batch['token_type_ids'].to(device)
  output=trained_model(input_ids,attention_mask,token_type_ids)
  input_ids=batch['input_ids'].to("cpu")
  attention_mask=batch['attention_mask'].to("cpu")
  token_type_ids=batch['token_type_ids'].to("cpu")
  for indi in output:
    op=torch.sigmoid(indi).cpu().detach().numpy().tolist()
    final_outputs.append(op)

In [ ]:
print(final_outputs)

[[0.9797741770744324, 0.03407011181116104, 0.12152023613452911, 0.009435010142624378, 0.01378452219069004, 0.005578425247222185, 0.004478250164538622, 0.005641293246299028, 0.006894278805702925], [0.04016959294676781, 0.012266375124454498, 0.9733319878578186, 0.014116724953055382, 0.010714010335505009, 0.005930523853749037, 0.00546798063442111, 0.007790419273078442, 0.014132163487374783], [0.04072597995400429, 0.01233619824051857, 0.9739295244216919, 0.01242203637957573, 0.011363894678652287, 0.006078130565583706, 0.005476450547575951, 0.0079722311347723, 0.0141255808994174], [0.09019000083208084, 0.9607883095741272, 0.9101737141609192, 0.020119957625865936, 0.012983663938939571, 0.005065198056399822, 0.0052077374421060085, 0.005613277200609446, 0.009675642475485802], [0.9808450937271118, 0.5255913138389587, 0.15350580215454102, 0.00665523624047637, 0.008764004334807396, 0.002148746745660901, 0.0027242128271609545, 0.0023034107871353626, 0.0028897032607346773], [0.47104692459106445, 0.

In [ ]:
print(len(final_outputs))
prediction_converted=[]
for prediction in final_outputs:
  indiarray=[]
  for indi in prediction:
    if indi>0.5:
      indiarray.append(1.0)
    else:
      indiarray.append(0.0)
  prediction_converted.append(indiarray)
print(prediction_converted)

0
[]


In [ ]:
count=0
total=0
for i in range(0,len(prediction_converted)):
  if prediction_converted[i]==encoded_dataset['test']["labels"][i]:
    count+=1
  total+=1
print("correct count",count)
print("total count",total)

correct count 2248
total count 3539
